# 0. Pipeline
* Import libararies
* Import Datasets
* Split the data into train_test
* Got idea about the input and output shape of the data
* Visualize the data and get insights of the data
* Segment the data into batch size by dataloader
* Got insight about the single batch of the data
* Create a baseline model
* Set Loss and Optimizer Function

In [1]:
# Import Libraries
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# TorchVision Libraries
from torchvision import datasets
from torchvision.transforms import ToTensor



In [11]:
# Train and Test Dataset
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download=True,
    target_transform = None
)
test_data =  datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download=True,
)

In [12]:
train_data,test_data

(Dataset MNIST
     Number of datapoints: 60000
     Root location: data
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 Dataset MNIST
     Number of datapoints: 10000
     Root location: data
     Split: Test
     StandardTransform
 Transform: ToTensor())

# 1. Input and Output shapes of Computer vision model

In [14]:
img,label = train_data[0]
img,label

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [40]:
# Shapes of input 
print(f"Shapes of input image: {img.shape} and Label: {label}")

# Number of Samples
print(f"Number of samples in train data: {len(train_data)}")
print(f"Number of samples in test data: {len(test_data)}")

Shapes of input image: torch.Size([1, 28, 28]) and Label: 2
Number of samples in train data: 60000
Number of samples in test data: 10000


In [15]:
train_data.class_to_idx

{'0 - zero': 0,
 '1 - one': 1,
 '2 - two': 2,
 '3 - three': 3,
 '4 - four': 4,
 '5 - five': 5,
 '6 - six': 6,
 '7 - seven': 7,
 '8 - eight': 8,
 '9 - nine': 9}

In [21]:
class_names = train_data.classes
class_names

['0 - zero',
 '1 - one',
 '2 - two',
 '3 - three',
 '4 - four',
 '5 - five',
 '6 - six',
 '7 - seven',
 '8 - eight',
 '9 - nine']

## Visualize the Data

(-0.5, 27.5, 27.5, -0.5)

In [24]:
plt.imshow(img.squeeze())
plt.title(class_names[label])
plt.axis(False)

In [37]:
# Randomly visualize some of the training data
# First of all I need randomly initiate index
import random
plt.figure(figsize = (9,9))
rows,cols = 4,4
random_idx = np.random.randint(0,len(train_data),16)
for idx,i in enumerate(random_idx):
    plt.subplot(4,4,idx+1)
    img,label = train_data[i]
    plt.imshow(img.squeeze())
    plt.title(class_names[label])
    plt.axis(False)
        


# 2. DataLoader (Prepare train and test Data)


In [41]:
from torch.utils.data.dataloader import DataLoader

# batch size
Batch_size = 32

# Turn datasets into iterable (batches)
train_dataloader = DataLoader(dataset = train_data,
                             batch_size = Batch_size,
                             shuffle=True)

test_dataloader = DataLoader(dataset = test_data,
                            batch_size=Batch_size,
                            shuffle = True)

In [42]:
train_dataloader,test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x30f03c050>,
 <torch.utils.data.dataloader.DataLoader at 0x30f18d790>)

In [58]:
train_dataloader.dataset,test_dataloader.dataset

(Dataset MNIST
     Number of datapoints: 60000
     Root location: data
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 Dataset MNIST
     Number of datapoints: 10000
     Root location: data
     Split: Test
     StandardTransform
 Transform: ToTensor())

In [51]:
print(f" Length of train data: {len(train_dataloader)} batches of 32")
print(f" Length of test data: {len(test_dataloader)} batches of 32")

 Length of train data: 1875 batches of 32
 Length of test data: 313 batches of 32


In [54]:
train_features_batch,train_features_label =next(iter(train_dataloader))
print(train_features_batch.shape)
print(train_features_label.shape)

torch.Size([32, 1, 28, 28])
torch.Size([32])


# 3. Create a Baseline model

In [60]:
from torch import nn

class MNISTModelV0(nn.Module):
    def __init__(self,
                input_image_shape:int,
                hidden_unit_shape:int,
                output_unit_shape:int):
        super().__init__()


        self.stack_model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features = input_image_shape,
                     out_features=hidden_unit_shape),
            nn.Linear(in_features = hidden_unit_shape,
                     out_features = output_unit_shape)
        )


        def forward(self,x):
            return self.stack_model(x)

In [61]:
## Device Agnostic Code
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [62]:
# Create a instance of the model and pass it to the device
model_0 = MNISTModelV0(input_image_shape=1,
                       hidden_unit_shape=10,
                       output_unit_shape=len(class_names)).to(device)

In [63]:
model_0

MNISTModelV0(
  (stack_model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [64]:
## Loss and Optimizer
loss_fn = nn.CrossEntropyLoss() # as it is a classification model

optimizer = torch.optim.Adam(model_0.parameters(),
                            lr = 0.01)


In [66]:
# Accuracy Function
from pathlib import Path
import requests

if Path("helper_functions.py").is_file():
    print("Helper Functions already available, Skip downloading.....")
else:
    request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
    with open("helper_functions.py","wb") as f:
        f.write(request.content)
        print("Download Complete")


Helper Functions already available, Skip downloading.....


In [68]:
from helper_functions import accuracy_fn

## Train and Test Loop Function

In [70]:
def train_loop(model: torch.nn.Module,
              data: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,optimizer:torch.optim.Optimizer,
              accuracy_fn,
              device):
    model.to(device)
    train_loss,accuracy = 0,0

    model.train()

    for X_train_batch,y_label in data:
        X_train_batch.to(device),y_label.to(device)

        # 1. Forward Pass
        y_logit = model(X_train)
        
        # Turn logit into pred_probs
        y_pred_label = torch.softmax(input = y_logit,
                              dim = 1).argmax(dim=1)
        
        # 2. Loss & Accuracy Calculation
        loss = loss_fn(y_logit,y_label) #y_logit is the loss of a whole batch --> basically loss of 32 images
        train_loss +=loss #accumulate the whole loss of the dataset
        acc = accuracy_fn(y_true = y_label,
                         y_pred=y_pred_label)
        accuracy +=acc

        # 3. Optimizer gradient zero
        optimizer.zero_grad()

        # 4. Backward Propagation
        loss.backward()

        # 5. gradient step
        optimizer.step()


    # Calculate the average loss and accuracy of the whole dataset and manifest them
    average_train_loss = train_loss/len(data)
    average_accuracy = accuracy / len(data)

    print(f"Total Train Loss: {average_train_loss:.3f} | Train Accuracy: {average_accuracy:.3f}")

In [71]:
def test_loop(model: torch.nn.Module,
             data: torch.utils.data.DataLoader,
             loss_fn: torch.nn.Module,
             accuracy_fn,
             device):
    model.to(device)
    test_loss,test_acc = 0,0

    model.eval()
    with torch.inference_mode():
        for X_test_batch,y_batch_label in data:
            X_test_batch.to(device),y_batch_label.to(device)
    
    
            # 1. Forward Pass
            y_test_logit = model(X_test_batch)
    
            # Turn logit into prediction probabilities
            y_pred_prob = torch.softmax(input = y_test_logit,
                                       dim =1).argmax(dim =1)
    
            # 2. Loss Calculation
            loss = loss_fn(y_test_logit,
                          y_batch_label)
            test_loss +=loss
            accuracy = accuracy_fn(y_true = y_batch_label,
                                  y_pred = y_pred_prob)
            test_acc +=accuray
    
        avg_test_loss = test_loss/len(data)
        avg_accuracy = test_acc/len(data)

        print(f"Test Loss: {avg_test_loss} | Accuracy: {avg_accuracy}")
            
            

## Evaluation Mode

In [ ]:
def eval